In [4]:
using GaussianProcesses, Plots, SumProductNetworks, StatsFuns, Distributions, StatsBase, DataFrames
import SumProductNetworks.add!

In [6]:
include("gpUtils.jl")

optimize2! (generic function with 1 method)

#### Data set

In [3]:
datapath = "../data/uci/energy.csv"

"../data/uci/energy.csv"

In [7]:
data = readtable("../data/uci/energy.csv");

In [164]:
X = Float64.(Array(data[[:X1, :X2, :X3, :X4, :X5, :X6, :X7, :X8]]));
X .-= minimum(X, 1)
X ./= maximum(X, 1);
X .-= mean(X, 1);


In [165]:
y = Array(data[[:Y1, :Y2]]);

y .-= mean(y, 1);

In [166]:
N = size(X, 1)

768

In [167]:
Ntrain = Int(ceil(N * 0.8))  # 1_000

srand(1234)

ids = collect(1:size(X, 1))
trainingIds = shuffle(ids)[1:Ntrain]
testingIds = shuffle(setdiff(ids, trainingIds));

In [168]:
Xtest = X[testingIds,:]
ytest = y[testingIds, :]

Xtrain = X[trainingIds, :]
ytrain = y[trainingIds, :];

In [169]:
mZero = MeanZero()
kern = SE(-1.0,0.0) 
logObsNoise = -1.0 

gp = GP(Xtrain', vec(ytrain[:,1]), mZero, kern, logObsNoise)
set_priors!(gp.k,[Normal(-2.0,4.0),Normal(-2.0,4.0)])

samples = mcmc(gp; nIter=100,burnin=0,thin=10);

BasicMCJob:
  Variable [1]: p (BasicContMuvParameter)
  GenericModel: 1 variables, 0 dependencies (directed graph)
  HMC sampler: number of leaps = 10, leap step = 0.1
  VanillaMCTuner: period = 100, verbose = false
  BasicMCRange: number of steps = 91, burnin = 0, thinning = 10

In [150]:
#Set the parameters to the posterior values the sample random function
fsamples = [];
for i in 1:size(samples,2)
    set_params!(gp, samples[:,i])
    update_target!(gp)
    μ, σ² = predict_y(gp, Xtest');
    push!(fsamples, μ)
end

In [151]:
μ = mean(reduce(hcat, fsamples), 2)

153×1 Array{Float64,2}:
 -7.79041
 12.1846 
  6.63563
 -8.71989
 -4.20884
 12.7829 
 -7.6357 
  2.82111
 17.2644 
 -9.74916
 13.2508 
 -8.76915
 10.7798 
  ⋮      
 -9.14533
 -6.8803 
 -4.88192
  1.89881
 18.1786 
 14.4113 
 15.1019 
 -9.9823 
 12.4038 
 -6.29291
 -8.18436
 -7.90336

In [209]:
mZero = MeanZero()
kern = SE(-1.0,0.0) 
logObsNoise = -1.0 

gp_1 = GP(Xtrain', vec(ytrain[:,1]), mZero, kern, logObsNoise)


μ, σ² = predict_y(gp_1, Xtrain');
#println(sqrt(mean((μ .- ytrain[:,1]).^2)))
#optimize2!(gp_1, mean = false, kern = true, noise = false, lik=false, lowerBound = -100)

μ, σ² = predict_y(gp_1, Xtrain');
println(sqrt(mean((μ .- ytrain[:,1]).^2)))

gp_2 = GP(Xtrain', vec(ytrain[:,2]), mZero, kern, logObsNoise)
#optimize2!(gp_2, mean = false, kern = true, noise = false, lik=false, lowerBound = -100)

1.0683756852622335


GP Exact object:
  Dim = 8
  Number of observations = 615
  Mean function:
    Type: GaussianProcesses.MeanZero, Params: Float64[]
  Kernel:
    Type: GaussianProcesses.SEIso, Params: [-1.0, 0.0]
  Input observations = 
[0.377315 -0.206019 … -0.206019 -0.344907; -0.368056 0.215278 … 0.215278 0.381944; … ; 0.0390625 -0.335938 … -0.335938 0.0390625; -0.1625 0.4375 … 0.0375 -0.1625]
  Output observations = [8.05224, -10.1978, -10.0478, 6.55224, -10.4478, 5.31224, -9.02776, -7.59776, -8.81776, -9.48776  …  -10.3178, -9.18776, -5.10776, 23.0022, -5.29776, 18.2722, 8.45224, -2.09776, -9.91776, -4.11776]
  Variance of observation noise = 0.1353352832366127
  Marginal Log-Likelihood = -8682.592

In [210]:
yhat = zeros(size(ytest))

μ, σ² = predict_y(gp_1, Xtest');
yhat[:,1] = μ

μ, σ² = predict_y(gp_2, Xtest');
yhat[:,2] = μ;

In [211]:
sqrt(mean((yhat .- ytest).^2))

2.0427949615273278

## Learn a full GP

In [11]:
mZero = MeanZero()                   #Zero mean function
kern = SE(0.0,0.0)                   #Sqaured exponential kernel (note that hyperparameters are on the log scale)

Type: GaussianProcesses.SEIso, Params: [0.0, 0.0]


In [12]:
logObsNoise = -1.0                        # log standard deviation of observation noise (this is optional)

-1.0

In [13]:
log(5)

1.6094379124341003

In [170]:
gp = GP(reshape(X, 1, N), y, MeanZero(), SE(log(5.0),log(1.0)), -2.)       #Fit the GP

LoadError: [91mDimensionMismatch("new dimensions (1, 768) must be consistent with array size 6144")[39m

In [171]:
μ, σ² = predict_y(gp,linspace(minimum(X),maximum(X),100));

LoadError: [91mArgumentError: Gaussian Process object and input observations do not have consistent dimensions[39m

In [16]:
scatter(X, y, label = "observations", title = "Full GP with SE Kernel and LLH: $(gp.target)")
plot!(linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "full GP with 95% interval")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 Full GP with SE Kernel and LLH: -125.00088180123245 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polygon clip-path="url(#clip7702)" points="
56.8551,355.642 66.1466,349.612 75.4382,347.459 84.7297,347.319 94.0213,348.015 103.313,349.073 112.604,350.409 121.896,352.04 131.188,353.885 140.479,355.686 
 149.771,357.039 159.062,357.504 168.354,356.741 177.645,354.62 186.937,351.288 196.228,347.182 205.52,343.009 214.812,339.709 224.103,338.394 233.395,340.268 
 242.686,346.518 251.978,358.169 261.269,375.934 270.561,400.063 279.852,430.248 289.144,465.587 298.436,504.637 307.727,545.544 317.019,586.221 326.31,624.538 
 335.602,658.503 344.893,686.403 354.185,706.907 363.476,719.124 372.768,722.629 382.059,717.448 391.351,704.024 400.643,683.154 409.934,655.903 419.226,623.506 
 428.517,587.275 437.809,548.504 447.1,508.412 456.392,468.097 465.683,428.531 474.975,390.563 484.267,354.947 493.558,322.354 502.85,293.388 512.141,268.576 
 521.433,248.349 530.724,233.019 540.016,222.749 549.307,217.539 558.599,217.212 567.891,221.421 577.182,229.637 586.474,241.159 595.765,255.109 605.057,270.453 
 614.348,286.052 623.64,300.758 632.931,313.532 642.223,323.569 651.515,330.392 660.806,333.901 670.098,334.353 679.389,332.309 688.681,328.54 697.972,323.935 
 707.264,319.408 716.555,315.824 725.847,313.92 735.139,314.232 744.43,317.018 753.722,322.215 763.013,329.435 772.305,338.017 781.596,347.131 790.888,355.916 
 800.179,363.604 809.471,369.625 818.763,373.659 828.054,375.64 837.346,375.728 846.637,374.253 855.929,371.655 865.22,368.423 874.512,365.031 883.803,361.887 
 893.095,359.272 902.387,357.29 911.678,355.838 920.97,354.608 930.261,353.155 939.553,351.045 948.844,348.077 958.136,344.534 967.427,341.35 976.719,340.004 
 976.719,173.941 967.427,188.343 958.136,198.094 948.844,204.51 939.553,208.728 930.261,211.556 920.97,213.547 911.678,215.126 902.387,216.665 893.095,218.49 
 883.803,220.826 874.512,223.737 865.22,227.074 855.929,230.477 846.637,233.431 837.346,235.361 828.054,235.747 818.763,234.235 809.471,230.709 800.179,225.322 
 790.888,218.468 781.596,210.732 772.305,202.804 763.013,195.401 753.722,189.188 744.43,184.708 735.139,182.322 725.847,182.149 716.555,184.023 707.264,187.48 
 697.972,191.802 688.681,196.099 679.389,199.438 670.098,200.976 660.806,200.067 651.515,196.326 642.223,189.643 632.931,180.175 623.64,168.311 614.348,154.646 
 605.057,139.957 595.765,125.173 586.474,111.337 577.182,99.541 567.891,90.8289 558.599,86.1073 549.307,86.068 540.016,91.149 530.724,101.533 521.433,117.177 
 512.141,137.85 502.85,163.181 493.558,192.692 484.267,225.814 474.975,261.906 465.683,300.247 456.392,340.039 447.1,380.398 437.809,420.354 428.517,458.847 
 419.226,494.734 409.934,526.81 400.643,553.842 391.351,574.632 382.059,588.103 372.768,593.402 363.476,590.012 354.185,577.855 344.893,557.352 335.602,529.447 
 326.31,495.549 317.019,457.435 307.727,417.095 298.436,376.573 289.144,337.804 279.852,302.476 270.561,271.931 261.269,247.091 251.978,228.409 242.686,215.857 
 233.395,208.942 224.103,206.768 214.812,208.155 205.52,211.783 196.228,216.366 186.937,220.804 177.645,224.287 168.354,226.34 159.062,226.801 149.771,225.764 
 140.479,223.506 131.188,220.45 121.896,217.125 112.604,214.133 103.313,212.062 94.0213,211.339 84.7297,212.01 75.4382,213.466 66.1466,214.189 56.8551,211.835 
 
 " fill="#e26f46" fill-opacity="0.5"/>
<polyline clip-path="url(#clip7702)" style="stroke:#e26f46; stroke-width:1; stroke-

In [17]:
yhat = reduce(vcat, [rand(gp, linspace(minimum(X),maximum(X),100)) for k in 1:100]);
yhat += rand() * 0.2
size(yhat)

(10000,)

In [18]:
μ, σ = predict_y(gp,linspace(minimum(X),maximum(X),100));
yhat = reduce(hcat, [rand(gp, linspace(minimum(X),maximum(X),100)) for k in 1:100]);

scatter(X, y, label = "observations", title = "Full GP with SE Kernel and LLH: $(gp.target)")
plot!(reduce(hcat, [linspace(minimum(X),maximum(X),100) for k in 1:100]), yhat, markersize = 2, markerstrokewidth = 0, legend =false)
plot!(linspace(minimum(X),maximum(X),100), μ + 2*σ, color = :green, w = 4, label = "Full GP mean + 2*std")
plot!(linspace(minimum(X),maximum(X),100), μ - 2*σ, color = :green, w = 4, label = "Full GP mean - 2*std")
savefig("../plots/full_gp_draws.png")

## optimize a full GP

In [19]:
optimize!(gp)

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [-2.0,1.6094379124341003,0.0]
 * Minimizer: [-1.2282746156639477,1.6156825479807913, ...]
 * Minimum: 3.510034e+01
 * Iterations: 12
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
   * |g(x)| < 1.0e-08: true
   * f(x) > f(x'): false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 56
 * Gradient Calls: 56

In [20]:
gp.target

-35.100343169513074

In [21]:
μ, σ² = predict_y(gp,linspace(minimum(X),maximum(X),100));

In [22]:
scatter(X, y, label = "observations", title = "Marginal LLH optimized full GP with SE Kernel and LLH: $(gp.target)")
plot!(linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "optimized full GP with 95% interval")
savefig("../plots/optimized_fullGP.png")

# Make a SPN with GP leaves

In [23]:
global gID = 0

0

Hand-coded 2-layer example

In [24]:
srand(12345678)
K = 10;
numSamples = 0.25;
meanFunction = MeanZero();
kernelFunction = SE(log(5.0),log(1.0));
noise = -1.;

In [29]:
root = GPSumNode(nextID(), Int[]);

for k in 1:K
    
    # make random (1D) split
    split = rand() * maximum(X) + minimum(X)
    n = min(sum(X .>= split), sum(X .< split))
    c = 0
    
    while n < (N * numSamples)
        @assert c < 100 "Could not find a split"
        split = rand() * maximum(X) + minimum(X)
        n = min(sum(X .>= split), sum(X .< split))
        c += 1
    end
    
    child = FiniteSplitNode(nextID(), Float64[split])
    
    # append two GP leaves
    leaf1 = GPLeaf{Any}(nextID(),
            GP(reshape(X[X .<= split], 1, sum(X .<= split)), y[X .<= split], meanFunction, kernelFunction, noise))
    leaf1.parents = SPNNode[]
    
    leaf2 = GPLeaf{Any}(nextID(),
            GP(reshape(X[X .> split], 1, sum(X .> split)), y[X .> split], meanFunction, kernelFunction, noise))
    leaf2.parents = SPNNode[]
    
    add!(child, leaf1)
    add!(child, leaf2)
    
    add!(root, child)
end

## Visualize splits

In [30]:
plt = scatter(X, y, label = "observations", title = "Random splits of SPN-GP")
plotSplits!(plt, getAllSplits(root))
savefig("../plots/randomSplits.png")

## Likelihood of product nodes

In [38]:

for (pi, pnode) in enumerate(root.children)
    
    μ, σ² = spn_predict(pnode, linspace(minimum(X),maximum(X),100));
    
    plt = scatter(X, y, label = "observations", title ="Prediction of product node $(pi), w: $(root.posterior_weights[pi]), LLH: $(spn_likelihood(pnode))")
    plot!(plt, linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "MoE GP with 95% interval")
    plotSplits!(plt, getAllSplits(pnode))
    savefig("../plots/productNode$(pi).png")

end

LoadError: [91mUndefVarError: spn_predict not defined[39m

## Compute likelihoods

In [39]:
# test
n = GPLeaf{Any}(0, GP(reshape(X, 1, N), y, meanFunction, kernelFunction, noise))
spn_likelihood(n)

-41.135026567609316

In [40]:
# test
n = root.children[3]
spn_likelihood(n)

-43.86502087845119

In [41]:
# test
spn_likelihood(root)

-42.66525472240039

## Update posterior weights

In [42]:
root.prior_weights = ones(length(root)) / length(root);

In [44]:
spn_update!(root)

-42.374565858981555

In [45]:
println(root)

Gaussian Process Sum Node [ID: 32, 
	 w_prior: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 
	 w_posterior: [0.109, 0.074, 0.023, 0.097, 0.126, 0.104, 0.237, 0.1, 0.097, 0.034]]


## Predictions under the model

In [46]:
# test
n = root.children[3]
μ, σ² = spn_predict(n, linspace(minimum(X),maximum(X),100));

plt = scatter(X, y, label = "observations", title ="Prediction of product node 3 only")
plot!(plt, linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "MoE GP with 95% interval")
plotSplits!(plt, getAllSplits(n))
savefig("../plots/productNode3.png")

LoadError: [91mUndefVarError: spn_predict not defined[39m

In [47]:
@assert sum(root.prior_weights) ≈ 1.
@assert sum(root.posterior_weights) ≈ 1.

In [48]:
# test
n = root
μ, σ² = spn_predict(n, linspace(minimum(X),maximum(X),100));

plt = scatter(X, y, label = "observations", title="SPN-GP with $(K) splits and LLH: $(spn_likelihood(root))")
plot!(plt, linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "SPN-GP with 95% interval")
plotSplits!(plt, getAllSplits(n))
savefig("../plots/spn_gp.png")

LoadError: [91mUndefVarError: spn_predict not defined[39m

## Print SPN

In [49]:
rDepth = depth(root)
for node in reverse(SumProductNetworks.order(root))
    print(repeat("\t", rDepth - depth(node)), " ", node, "\n")
end

LoadError: [91mUndefVarError: order not defined[39m

## Create SPN-GP with flexible depth

In [50]:
K = 5;
numSamples = 0.25;
minSamples = 40;
meanFunction = MeanZero();
kernelFunction = SE(log(5.0),log(1.0));
noise = -1.;
maximumDepth = 1

param = SPNParameters(K, numSamples, minSamples, kernelFunction, meanFunction, noise)

root = makeSumNode(X, y, param, 1, maximumDepth)

LoadError: [91mUndefVarError: SPNParameters not defined[39m

In [595]:
spn_update(root)

-42.29047283482476

In [596]:
spn_likelihood(root)

-42.29047283482476

In [570]:
root

Gaussian Process Sum Node [ID: 3051258, 
	 w_prior: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 
	 w_posterior: [0.181, 0.028, 0.08, 0.134, 0.176, 0.016, 0.026, 0.071, 0.198, 0.09]]

In [571]:
depth(root)

4

In [572]:
sum(isa.(SumProductNetworks.order(root), SumNode))

21

In [525]:
# test
n = root
μ, σ² = spn_predict(n, linspace(minimum(X),maximum(X),100));


plt = scatter(X, y, label = "observations", title="SPN-GP with depth $(depth(root)) and $(K) splits and LLH: $(spn_likelihood(root))")
plot!(plt, linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "SPN-GP with 95% interval")
#plotSplits!(plt, getSplits(n, 1))
savefig("../plots/deep_spn_gp.png")

plt = scatter(X, y, label = "observations", title="SPN-GP with depth $(depth(root)) and $(K) splits and LLH: $(spn_likelihood(root))")
plot!(plt, linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "SPN-GP with 95% interval")
plotSplits!(plt, getSplits(n, 1))
savefig("../plots/deep_spn_gp_splits.png")

In [500]:
include("../../infinitesumproductnetworks/src/plotSPN.jl")

spn2graphviz (generic function with 1 method)

In [552]:
print(root)

Gaussian Process Sum Node [ID: 3035894, 
	 w_prior: [0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02], 
	 w_posterior: [0.03, 0.03, 0.026, 0.019, 0.029, 0.007, 0.014, 0.012, 0.022, 0.007, 0.007, 0.009, 0.032, 0.028, 0.006, 0.017, 0.016, 0.007, 0.016, 0.023, 0.032, 0.021, 0.018, 0.028, 0.019, 0.085, 0.006, 0.008, 0.029, 0.007, 0.006, 0.018, 0.029, 0.03, 0.03, 0.026, 0.02, 0.007, 0.007, 0.03, 0.027, 0.021, 0.018, 0.006, 0.034, 0.027, 0.008, 0.007, 0.022, 0.017]]

## some plots

#### Flat SPN

In [652]:
K = 10000;
numSamples = 0.25;
minSamples = 40;
meanFunction = MeanZero();
kernelFunction = SE(log(5.0),log(1.0));
noise = -1.;
maximumDepth = 1

param = SPNParameters(K, numSamples, minSamples, kernelFunction, meanFunction, noise)

root = makeSumNode(X, y, param, 1, maximumDepth)
spn_update(root)

-42.75693752401281

In [ ]:
for K in [5, 50, 500, 5_000, 10_000, 50_000]

    numSamples = 0.25;
    minSamples = 40;
    meanFunction = MeanZero();
    kernelFunction = SE(log(5.0),log(1.0));
    noise = -1.;
    maximumDepth = 3

    param = SPNParameters(K, numSamples, minSamples, kernelFunction, meanFunction, noise)

    root = makeSumNode(X, y, param, 1, maximumDepth)
    spn_update(root)

    S = 100

    x = linspace(minimum(X),maximum(X),100)
    xx = reduce(hcat, x for k in 1:S)
    yy = reduce(hcat, spn_rand(root, x) for k in 1:S)

    μ, σ = predict_y(gp,linspace(minimum(X),maximum(X),100));

    plt = scatter(X, y, label = "observations", legend = false)
    plot!(xx, yy, markersize = 2, markerstrokewidth = 0, label = "Posterior samples from SPN-GP");
    plot!(linspace(minimum(X),maximum(X),100), w =4, μ + 2*σ, color = :green, label = "Full GP mean + 2*std")
    plot!(linspace(minimum(X),maximum(X),100), w =4, μ - 2*σ, color = :green, label = "Full GP mean - 2*std")
    
    splits = [child.split[1] for child in children(root)]
    
    vline!([minimum(splits), maximum(splits)], color = :yellow, w = 4)

    plot(plt, size = (1024, 1024), title="SPN-GPs with differen $(K) splits")
    savefig("../plots/spn_gp_splits_$(K)_$(maximumDepth).png")
    
end

In [670]:
plot(
histogram([child.split[1] for child in children(root)], weightss = root.posterior_weights, title= "posterior weighted"), 
histogram([child.split[1] for child in children(root)], title = "prior over splits"))
savefig("../plots/split_weights.png")

#### Deep SPN

In [625]:
K = 5;
numSamples = 0.25;
minSamples = 40;
meanFunction = MeanZero();
kernelFunction = SE(log(5.0),log(1.0));
noise = -1.;
maximumDepth = 100

param = SPNParameters(K, numSamples, minSamples, kernelFunction, meanFunction, noise)

root = makeSumNode(X, y, param, 1, maximumDepth)
spn_update(root)

-47.93831375543295

In [631]:
S = 100

x = linspace(minimum(X),maximum(X),100)
xx = reduce(hcat, x for k in 1:S)
yy = reduce(hcat, spn_rand(root, x) for k in 1:S)

μ, σ = predict_y(gp,linspace(minimum(X),maximum(X),100));

plt1 = scatter(X, y, label = "observations", legend = false)
plot!(xx, yy, markersize = 2, markerstrokewidth = 0, label = "Posterior samples from SPN-GP");
plot!(linspace(minimum(X),maximum(X),100), μ + 2*σ, color = :green, label = "Full GP mean + 2*std")
plot!(linspace(minimum(X),maximum(X),100), μ - 2*σ, color = :green, label = "Full GP mean - 2*std")

plt2 = scatter(X, y, label = "observations")
plot!(xx, yy, markersize = 2, markerstrokewidth = 0, label = "Posterior samples from SPN-GP");
plot!(linspace(minimum(X),maximum(X),100), μ + 2*σ, color = :green, label = "Full GP mean + 2*std")
plot!(linspace(minimum(X),maximum(X),100), μ - 2*σ, color = :green, label = "Full GP mean - 2*std")
plotSplits!(plt2, getAllSplits(root))

plot(plt1, plt2, size = (2024, 768), title="SPN-GP with depth $(depth(root) - 1) and $(K) splits per sum node (LLH: $(spn_likelihood(root)))")
savefig("../plots/spn_gp_deep.png")